Домашнее задание
1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
2. сделать feature engineering
3. обучить любой классификатор (какой вам нравится)
4. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
5. применить random negative sampling для построения классификатора в новых условиях
6. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
7. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import catboost as catb
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

In [ ]:
df = pd.read_csv('weatherAUS.csv', sep=',')
df.head(3)

In [ ]:
df.shape

In [ ]:
df.describe()

In [ ]:
# Посомтрим на пропуски

df.isna().sum()

In [ ]:
# Посмотрим на типы данных в датасете

df.dtypes

In [ ]:
target_name = 'RainTomorrow'
base_feature_names = df.columns.drop([target_name]).tolist()
cat_feature_names = df.select_dtypes(include='object').columns.drop([target_name]).tolist()
num_feature_names = df.columns.drop([target_name] + cat_feature_names).tolist()

In [ ]:
df['RainTomorrow'].unique()

In [ ]:
# преобразуем target

df.loc[df['RainTomorrow'] == 'No', 'RainTomorrow'] = 0
df.loc[df['RainTomorrow'] == 'Yes', 'RainTomorrow'] = 1

In [ ]:
df['RainTomorrow'].unique()

In [ ]:
df['RainTomorrow'].isna().sum()

In [ ]:
#выкинем  значения, где наш таргет неопределен

df = df[df['RainTomorrow'].notna()]

In [ ]:
# поменяем тип данных

df = df.astype({"RainTomorrow": int})

In [ ]:
#Посмотрим на соотношение классов 

f, ax = plt.subplots(figsize=(6, 6))
ax = sns.countplot(x="RainTomorrow", data=df, palette="Set1")
plt.show()

In [ ]:
#Посмотрим на соотношение классов в численном выражении

df['RainTomorrow'].value_counts()

### Корреляция целевой переменной с базовыми признаками

In [ ]:
corr_with_target = df[base_feature_names + [target_name]].corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(10, 8))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Корреляция целевой переменной с базовыми признаками')
plt.show()

### Матрица корреляций признаков

In [ ]:
plt.figure(figsize = (15,15))

sns.set(font_scale=1.4)

corr_matrix = df[base_feature_names].corr()
corr_matrix = np.round(corr_matrix, 2)
corr_matrix[np.abs(corr_matrix) < 0.2] = 0

sns.heatmap(corr_matrix, annot=True, linewidths=.5, cmap='GnBu')

plt.title('Матрица корреляций признаков')
plt.show()

### Анализ выбросов

In [ ]:
df[num_feature_names].hist(figsize=(20, 20), bins=80, grid=False);

In [ ]:
# посомтрим на сводную статистику в числовыхпеременных

print(round(df[num_feature_names].describe()),2)

In [ ]:
# обработаем пропущенные значения числовых признаков в X_train

for col in df[num_feature_names]:
    col_median = df[col].median()
    df[col].fillna(col_median, inplace=True) 

In [ ]:
# проверим наличие пропусков еще раз X_train

df[num_feature_names].isnull().sum()

In [ ]:
# проверка количества элементов в категориальных переменных

for var in cat_feature_names:
    
    print(var, ' содержит ', len(df[var].unique()), ' labels')

Мы видим, что есть переменная Date, которую нужно предварительно обработать. Предварительную обработку сделаем позже.

In [ ]:
# приведем фичу Date к нормальному формату даты 

df['Date'] = pd.to_datetime(df['Date'])

In [ ]:
# извлечь год, месяц и день из числа

df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day

In [ ]:
# удалим столбец Date

df.drop('Date', axis=1, inplace = True)

In [ ]:
df.head(3)

In [ ]:
df.dtypes

In [ ]:
# Обновим наши список наших категориальных фичей, чтобы в нем не было даты, которая была типом "object"
cat_feature_names = df.select_dtypes(include='object').columns.tolist()
cat_feature_names

In [ ]:
# вывести процент пропущенных значений в категориальных переменных в X_train

df[cat_feature_names].isnull().mean()

In [ ]:
# присвоим отсутствующим категориальным переменным наиболее частое значение

df['WindGustDir'].fillna(df['WindGustDir'].mode()[0], inplace=True)
df['WindDir9am'].fillna(df['WindDir9am'].mode()[0], inplace=True)
df['WindDir3pm'].fillna(df['WindDir3pm'].mode()[0], inplace=True)
df['RainToday'].fillna(df['RainToday'].mode()[0], inplace=True)

In [ ]:
# переведем признак RainToday в числовой
df.loc[df['RainToday'] == 'No', 'RainToday'] = 0
df.loc[df['RainToday'] == 'Yes', 'RainToday'] = 1

In [ ]:
df['RainToday'].unique()

In [ ]:
df = df.astype({"RainToday": int})

In [ ]:
# # Обновим наши список наших категориальных фичей, чтобы в нем не было RainToday
cat_feature_names = df.select_dtypes(include='object').columns.tolist()
cat_feature_names

In [ ]:
df

In [ ]:
#кодируем все категориальные признаки с помощью get_dummies
df = pd.get_dummies(df, columns = cat_feature_names, prefix_sep = "_", drop_first = True) 

In [ ]:
# удаляем закодированные категориальные признаки
df = df.drop(columns=cat_feature_names)
df.head()

In [ ]:
X = df.drop(columns=target_name)
y = df[target_name]

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.3, shuffle=True, random_state=42)

In [ ]:
X_train.shape, X_valid.shape, y_train.shape, y_valid.shape

In [ ]:
catb_model = catb.CatBoostClassifier(
    random_state=42,  
    n_estimators=1000, 
    learning_rate=0.025,
#     cat_features = cat_feature_names
)

catb_model.fit(X_train, y_train)
y_predict = catb_model.predict(X_valid)

In [ ]:
# посмоттрим на метрики

results = {}

def evaluate_results(y_valid, y_predict):

    f1 = f1_score(y_valid, y_predict)
    roc = roc_auc_score(y_valid, y_predict)
    rec = recall_score(y_valid, y_predict, average='binary')
    prc = precision_score(y_valid, y_predict, average='binary')
    results = {'metric': ['f1_score', 'roc_auc_score', 'recall', 'precision'],
               'catboost_model': [f1, roc, rec, prc],
              }

    return results

In [ ]:
results_catb_model = evaluate_results(sample_test.iloc[:,-2].values, y_predict)
results_catb_model

In [ ]:
# result_df = pd.DataFrame(evaluate_results(y_valid, y_predict))
# result_df

In [ ]:
# скопируем наш датасет
mod_df = df.copy()

#добавим тестовый признак
mod_df['class_test'] = 0

# беру только RainTomorrow = 1
mod_df_P = mod_df.loc[mod_df['RainTomorrow'] == 1]
mod_df_N = mod_df.loc[mod_df['RainTomorrow'] == 0]

# делю датафрейм 70 на 30 и перемешиваю его 
mod_df_P_70, mod_df_P_30 = train_test_split(mod_df_P, test_size=0.3, shuffle=True, random_state=42)

# беру кусочек 30% и заменяю в нем значения class_test на 1, там где RainTomorrow = 1
mod_df_P_30.loc[mod_df_30['RainTomorrow'] == 1, 'class_test'] = 1
mod_df_P_30.head(5)

In [ ]:
mod_df_N.shape, mod_df_P_70.shape, mod_df_P_30.shape

In [ ]:
mod_df = pd.concat([mod_df_P_30, mod_df_P_70, mod_df_N],axis=0)
mod_df

In [ ]:
# поменяем расстановку столбцов для удобства

mod_df = mod_df[[c for c in mod_df if c not in ['RainTomorrow', 'class_test']] 
       + ['RainTomorrow', 'class_test']]
mod_df

In [ ]:
mod_df = mod_df.sample(frac=1)
neg_sample = mod_df[mod_df['class_test']==0][:len(mod_df[mod_df['class_test']==1])]
sample_test = mod_df[mod_df['class_test']==0][len(mod_df[mod_df['class_test']==1]):]
pos_sample = mod_df[mod_df['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

In [ ]:
model = catb.CatBoostClassifier(
    random_state=42,  
    n_estimators=1200, 
    learning_rate=0.025,
#     cat_features = cat_feature_names
)

In [ ]:
model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)

In [ ]:
# посмоттрим на метрики

results = {}

def evaluate_results_rns(y_valid, y_predict):

    f1 = f1_score(y_valid, y_predict)
    roc = roc_auc_score(y_valid, y_predict)
    rec = recall_score(y_valid, y_predict, average='binary')
    prc = precision_score(y_valid, y_predict, average='binary')
    results = {'metric': ['f1_score', 'roc_auc_score', 'recall', 'precision'],
               'rns_model': [f1, roc, rec, prc],
              }

    return results

In [ ]:
results_rns_model = evaluate_results_rns(sample_test.iloc[:,-2].values, y_predict)
models_results['rns_model'] = results_rns_model['rns_model']

In [ ]:
result_df = pd.DataFrame(models_results)
result_df